In [120]:
class TrafficData:
    def __init__(self, company, year, month, cell_id, band, band_number, bandwidth, address, BT, RB, Connection, AC):
        self.company = company
        self.year = year
        self.month = month
        self.cell_id = cell_id
        self.band = band
        self.band_number = band_number
        self.bandwidth = bandwidth
        self.address = address
        self.BT = BT
        self.RB = RB
        self.Connection = Connection
        self.AC = AC

In [173]:
#-*- coding: utf-8 -*-
import xlrd
import os
import pandas as pd
import numpy as np
import matplotlib
from plotnine import *
from matplotlib import font_manager, rc
import math

col_alphabet=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','AA']
def file_read(file):
    #엑셀의 column index 배열
    check_index=-1 # error checking을 위한 index
    workbook = xlrd.open_workbook(os.path.join('/Users/sjlee/Desktop/Graduate/intern_etri/Preprocess/2017', file), encoding_override="utf-8")
    #sheetNumber = workbook.nsheets # sheet 개수
    #company_data = Company_data(None, None, [], [])
    trafficdata_list = []
    for index in range(0,5):
        worksheet = workbook.sheet_by_index(index)  # index를 이용해 sheet load
        worksheetName = worksheet.name  # load한 sheet이 이름, 특정 sheet에서 작업 시작
        #if worksheetName.startswith('4G'):  # 1)유효한 sheet name인지 check
        row_index = 0  #excel의 row줄에 대한 index
        company_name = worksheet.cell_value(0,1)  #'B1' 셀 값 load; cell_value(row_number, col_number)
        measurements = worksheet.cell_value(1,1)  # 대상 셀 기준
        cell_number = worksheet.cell_value(2,1)  # 'B3' 셀 값 load; 총 cell 갯수

        if(isinstance(cell_number, float)):  # 2) cell 갯수가 숫자 형식인가 check
            cell_number = int(cell_number)  #cell 갯수 int형으로 변환
            period = worksheet.cell_value(3,1)  #'B4' 셀 값 load; 측정 기간
            split_period1 = period.split("-")  # 측정 기간이 '-'로 표기된 경우: '-'를 이용하여 split
            split_period2 = period.split("~")  # 측정 기간이 '~'로 표기된 경우: '~'를 이용하여 split

            if (len(split_period1)==2 or len(split_period2)==2):  # 3) 'B4' 셀의 측정기간의 형식이 적절한가
                if len(split_period1) == 2:  # '-'을 기준으로 split된 경우
                    parse_date_start = split_period1[0].split(".")  #'.'을 기준으로 split; 측정 날짜 형식이 yyyy.MM.dd - yyyy.MM.dd
                    parse_date_end = split_period1[1].split(".")
                    end_day = split_period1[1].split('.')[2]  # yyyy.MM.dd 에서 dd를 가져옴 (측정 종료날)
                else: # '~'을 기준으로 split된 경우
                    parse_date_start = split_period2[0].split(".")  # '.'을 기준으로 split; 측정 날짜 형식이 yyyy.MM.dd - yyyy.MM.dd
                    parse_date_end = split_period2[1].split(".")
                    end_day = split_period2[1].split('.')[2]  # yyyy.MM.dd 에서 dd를 가져옴 (측정 종료날)
                year=parse_date_start[0] #측정 년도
                month=parse_date_start[1] #측정 월
                start_day=parse_date_start[2] #측정 시작날
                period_length=int(end_day)-int(start_day)+1	 #측정 기간 check

                ### 4) 측정 시작 날과 종료 날의 달이 다를 경우 check
                if parse_date_start[1] == parse_date_end[1]:
                    band_value=worksheet.cell_value(8,1) # 'B9' 셀 값 load; 대역

                    if(band_value<1000) : # 대역값이 1000보다 작은 경우
                        if(band_value<10): # 10보다 작은 경우는 GHz 단위를 붙임
                            band=str(band_value)+'GHz'
                        else :  # 10보다 큰 경우는 MHz 단위를 붙임
                            band=str(band_value)+'MHz'
                    else : # 1000으로 나눠준 뒤 GHz 단위를 붙임
                        band=str(band_value/1000.0)+'GHz'
                    bandwidth_value=worksheet.cell_value(9,1)
                    bandwidth=str(bandwidth_value)+'MHz'
                    band_number = index + 1
                    band = band+"({})".format(round(bandwidth_value))

                    data_start_row_index=14 # 데이터 시작 row index
                    for row_index in range(0,cell_number):  #row 방향으로 data 검사
                        cur_row_index = data_start_row_index + row_index # 현재 접근 중인 row index (row_index에 14를 더한 값)
                        cell_id = worksheet.cell_value(cur_row_index,0)
                        eNB_id=str((worksheet.cell_value(cur_row_index,1)))+str((worksheet.cell_value(cur_row_index,2))) # enb_id load
                        #eNB_id = str(eNB_id).decode('ascii')
                        address=worksheet.cell_value(cur_row_index,4) # address load
                        address_gu=str(address).split(" ")[1] # address 중 '구' 데이터
                        # cell_id, address data의 유효성 검사 유효하지 않으면 해당 row는 ignore
                        #8. eNB id data가 유효한가
                        if(check_invalid_data(eNB_id)):
                            #print ("Check eNB_id data:", "(",row_index, "1)")
                            #error_data["eNB_id"].append("{}".format(getExcelCellIndex(row_index,1)))  ##일단은 string으로!
                            continue
                        # 9. 주소 data가 유효한가
                        if(check_invalid_data(address)):
                            #print ("Check address data:", "(",row_index, "4)")
                            #error_data["address"].append("{}".format(getExcelCellIndex(row_index,4)))
                            continue

                        ######cell_data['address'] = address  # address; '구'만 저장할지 고민?
                        col_start_index=7 # column start index 월~금 데이터를 읽어오기 위한 index
                        chk_valid=0
                        tmp_Traffic=[]
                        tmp_RB=[]
                        tmp_AC_Traffic=[]
                        tmp_connection=[]

                        for col_index in range(0,period_length):  #col 방향으로 data 검사
                            #day = 1
                            DL_traffic = worksheet.cell_value(cur_row_index,col_start_index) # DL traffic data load
                            col_start_index += 1 # column 증가; 엑셀에서 한칸 옆으로
                            RB = worksheet.cell_value(cur_row_index,col_start_index) # RB data load
                            col_start_index += 1
                            connection= worksheet.cell_value(cur_row_index,col_start_index) # 호 접속수 data load
                            col_start_index += 1
                            AC_DL_traffic=worksheet.cell_value(cur_row_index,col_start_index) # 일누적 DL traffic data load
                            col_start_index += 1
                            #data_day=str(int(start_day)+col_index) #며칠의 데이터인지
                            # 12. DL Traffic data가 유효한가 (invalid -> break)
                            if(check_invalid_data(DL_traffic)):
                                #error_data["DL_traffic"].append("{}".format(getExcelCellIndex(cur_row_index,col_start_index-4)))
                                break
                            # 13. RB data가 유효한가
                            if(check_invalid_data(RB)):
                                #error_data["RB_Use"].append("{}".format(getExcelCellIndex(cur_row_index,col_start_index-3)))
                                break
                            # 14. 호접속수 data가 유효한가
                            if(check_invalid_data(connection)):
                                #error_data["Max_call"].append("{}".format(getExcelCellIndex(cur_row_index,col_start_index-2)))
                                break
                            # 15. 일누적 DL Traffic data가 유효한가
                            if(check_invalid_data(AC_DL_traffic)):
                                #error_data["AC_traffic"].append("{}".format(getExcelCellIndex(cur_row_index,col_start_index-1)))
                                break
                            chk_valid+=1
                            tmp_Traffic.append(DL_traffic)  # cell ID 별 DL_traffic
                            tmp_AC_Traffic.append(AC_DL_traffic)
                            tmp_RB.append(RB)
                            tmp_connection.append(connection)
                            #print(tmp_Traffic)

                        if(chk_valid==5) :  # 월화수목금 중에 한 칸이라도 invalid하면 chk_valid 값이 5보다 작으므로 그 row를 없앰
                            trafficdata = TrafficData(company_name, year, month, cell_id, band, band_number, bandwidth_value, address_gu, np.mean(tmp_Traffic), np.mean(tmp_RB), np.mean(tmp_connection), np.mean(tmp_AC_Traffic))
                            #trafficdata = TrafficData(company_name, year, month, cell_id, address_gu, band, bandwidth_value, day, tmp_Traffic[i], tmp_RB[i], tmp_connection[i], tmp_AC_Traffic[i])
                            trafficdata_list.append(trafficdata)
            print(trafficdata_list[0].band)
                            
    return trafficdata_list


    


###########class로 짜기!

def check_invalid_data(data):
    if data==42:
        if isinstance(data,float):
            return False
        else :
            return True
    if data=='' or data=='-' or data==0 : # ==42 는 #N/A 값 검사. #N/A 값을 load하면 42로 읽힘
        #print(data)
        return True
    else :
        return False

def getExcelCellIndex(row,col):
    CellIndex=col_alphabet[col]+str(row+1)
    return CellIndex

band_list = ['800.0MHz(10)','1.8GHz(20)', '2.1GHz(10)','2.6GHz(20)', '2.6GHz(10)']
trafficdata_list = file_read('201712_SKT.xlsx')

800.0MHz(10)
800.0MHz(10)
800.0MHz(10)
800.0MHz(10)
800.0MHz(10)


In [174]:
def makeBaseDf(trafficdata_total):
    result_Traffic = {"band": [],  "band_number": [],"bandwidth": [], "RB": []}
    month = trafficdata_total[0].month
    for i in range(0, len(trafficdata_total)):
        result_Traffic["band"].append(trafficdata_total[i].band)
        result_Traffic["band_number"].append(trafficdata_total[i].band_number)
        result_Traffic["bandwidth"].append(trafficdata_total[i].bandwidth)
        result_Traffic["RB"].append(math.ceil(trafficdata_total[i].RB))
    result_df = pd.DataFrame(result_Traffic)
    return month, result_df



def SaveExcel(month, XY_list):
    writer_orig = pd.ExcelWriter('/Users/sjlee/Desktop/Graduate/intern_etri/RBdistribution/Basic_{}.xlsx'.format(month), engine='openpyxl')
    for i in range(0, round(len(XY_list)/3)):
        band = band_list[i]
        j = 3*i
        XY_data = {"RB_{}".format(band):XY_list[j], "RB_tan_{}".format(band): XY_list[j+1], "prob_{}".format(band):XY_list[j+2]}
        XY_df = pd.DataFrame(XY_data)
        XY_df.to_excel(writer_orig, startcol = j+2, index=False, sheet_name = "sheet1")
    writer_orig.save()

def makeXY(month, df):
    month = month
    XY_list = []
    for i in range(0,len(band_list)):
        band = band_list[i]
        subdf = df[df["band"]==band]
        total_cell = subdf.band.count()
        RB_list = np.unique(subdf["RB"].values.tolist())
        #print(np.unique(subdf["RB"].values.tolist()))
        RB_list = RB_list[1:]
        #print(RB_list)
        X_list = np.tan(math.pi*RB_list/100.0 - math.pi/2)
        Y_list = []
        for j in range(0,len(RB_list)):
            temp_RB = RB_list[j]
            RB_count = subdf[subdf["RB"] == temp_RB].count().RB
            Y_list.append(RB_count)
        Y_list = Y_list/(total_cell*10)
        XY_list.append(RB_list)
        XY_list.append(X_list)
        XY_list.append(Y_list)
    SaveExcel(month, XY_list)
    print(len(XY_list))


print(len(trafficdata_list))
month, result_frame= makeBaseDf(trafficdata_list)
makeXY(month, result_frame)
            

2744
15
